In [1]:
import os
import yaml


# 2. Định nghĩa đường dẫn đến file data.yaml
# Lưu ý: Giữ nguyên cách viết "AI Scienctist" theo đúng file của bạn
yaml_path = 'data_rip.yaml'

# 3. Kiểm tra file data.yaml
if os.path.exists(yaml_path):
    print(f"\n✅ Đã tìm thấy file data.yaml tại:\n   {yaml_path}")
    
    # Mở và đọc nội dung file
    with open(yaml_path, 'r', encoding='utf-8') as f:
        data_config = yaml.safe_load(f)
        
    print("\n--- Nội dung đang đọc từ data.yaml ---")
    print(yaml.dump(data_config, allow_unicode=True, default_flow_style=False))
    print("--------------------------------------\n")
    
    # 4. Kiểm tra thư mục chứa ảnh từ đường dẫn trong data.yaml
    base_path = data_config.get('path', '')
    train_dir = os.path.join(base_path, data_config.get('train', ''))
    val_dir = os.path.join(base_path, data_config.get('val', ''))
    
    print("Kiểm tra thư mục dataset:")
    
    # Kiểm tra Train
    if os.path.exists(train_dir):
        train_files = len(os.listdir(train_dir))
        print(f"✅ Thư mục Train tồn tại. Có {train_files} file/thư mục con bên trong.")
    else:
        print(f"❌ Không tìm thấy thư mục Train tại:\n   {train_dir}")
        
    # Kiểm tra Val
    if os.path.exists(val_dir):
        val_files = len(os.listdir(val_dir))
        print(f"✅ Thư mục Val tồn tại. Có {val_files} file/thư mục con bên trong.")
    else:
        print(f"❌ Không tìm thấy thư mục Val tại:\n   {val_dir}")

else:
    print(f"❌ KHÔNG tìm thấy file. Hãy kiểm tra lại chính tả của đường dẫn:\n   {yaml_path}")


✅ Đã tìm thấy file data.yaml tại:
   data_rip.yaml

--- Nội dung đang đọc từ data.yaml ---
names:
- rip_current
nc: 1
path: ./datasets
train: images/train
val: images/val

--------------------------------------

Kiểm tra thư mục dataset:
✅ Thư mục Train tồn tại. Có 17387 file/thư mục con bên trong.
✅ Thư mục Val tồn tại. Có 4347 file/thư mục con bên trong.


## Tune parameter

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

def tune_model():
    # Khởi tạo mô hình Nano để tune (nhanh hơn bản Small)
    model = YOLO("yolo26n-seg.pt")

    print("Bắt đầu quá trình Hyperparameter Tuning...")
    # Quá trình này sẽ sinh ra các biến thể tham số và kiểm thử
    model.tune(
        data="data_rip.yaml",
        epochs=30,          # Số epoch cho mỗi lần thử nghiệm
        iterations=50,      # Số lần thử nghiệm các bộ tham số khác nhau
        optimizer="AdamW",
        plots=False,
        save=False,
        val=False
    )
    print("Tuning hoàn tất. File tham số tốt nhất được lưu tại: runs/segment/tune/best_hyperparameters.yaml")

if __name__ == "__main__":
    tune_model()

Bắt đầu quá trình Hyperparameter Tuning...
Tuner: Initialized Tuner instance with 'tune_dir=/home/goosevb/code/urlab/test_model/runs/segment/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/50 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}
New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.10 🚀 Python-3.10.19 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3768MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, b

## Training Part

In [ ]:
from ultralytics import YOLO

def train_models():
    # --- Huấn luyện YOLO26 Nano (Ưu tiên cho Jetson) ---
    print("Đang huấn luyện YOLO26n-seg...")
    model_n = YOLO("yolo26n-seg.pt")
    
    # Bạn có thể truyền file tham số đã tune vào đây bằng đối số `cfg` nếu muốn, 
    # hoặc để nó tự động học theo mặc định.
    results_n = model_n.train(
        data="data.yaml",
        epochs=100,
        imgsz=640,          # Có thể giảm xuống 416 nếu GPU của bạn bị quá tải
        batch=16,           # Điều chỉnh tùy thuộc vào VRAM của máy train
        project="rip_current_project",
        name="yolo26n_train",
        # Các thôngdata.yaml số mAP, Recall, Loss sẽ tự động in ra terminal và lưu vào file results.csv
    )

    # # --- Huấn luyện YOLO26 Small (Giả thuyết dự phòng) ---
    # print("Đang huấn luyện YOLO26s-seg...")
    # model_s = YOLO("yolo26s-seg.pt")
    # results_s = model_s.train(
    #     data="/home/goosevb/code/urlab/test_model/data.yaml",
    #     epochs=100,
    #     imgsz=640,
    #     batch=16,
    #     project="rip_current_project",
    #     name="yolo26s_train",
    # )

if __name__ == "__main__":
    train_models()